In [ ]:
import re
import os
import time
import json
import PyPDF2
import pandas as pd
from tqdm import tqdm
import lxml.etree as ET

from langchain.text_splitter import RecursiveCharacterTextSplitter

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

## Utility functions

In [ ]:
isLinux = True
default_linux_path = os.getcwd().replace("/Data", "/Documents/Downloaded") if "/Data" in os.getcwd() else os.getcwd() + "/Documents/Downloaded"
default_windows_path = os.getcwd().replace("\\Data", "\\Documents\\Downloaded") if "\\Data" in os.getcwd() else os.getcwd() + "\\Documents\\Downloaded"
default_path = default_linux_path if isLinux else default_windows_path

DEFAULT_SAVE_DIR = default_path.replace("/Downloaded", "/Generated") if isLinux else default_path.replace("\\Downloaded", "\\Generated")
CHROME_DRIVERS_PATH = "/home/giacomo/chromedriver-linux64/chromedriver" if isLinux else "C:\\Users\\giaco\\Downloads\\chromedriver-win64\\chromedriver.exe"
# Unibo: /chromedriver-linux64/chromedriver or /home/antonelli2/chromedriver-linux64/chromedriver
# WSL: /home/giacomo/chromedriver-linux64/chromedriver
CODICE_PENALE_PDF = default_path + ("/Codice penale well formatted edited.pdf" if isLinux else "\\Codice penale well formatted edited.pdf")
CODICE_PENALE_CSV = DEFAULT_SAVE_DIR + ("/Codice penale well formatted edited.csv" if isLinux else "\\Codice penale well formatted edited.csv")

CPP_CSV = DEFAULT_SAVE_DIR + ("/Codice procedura penale.csv" if isLinux else "\\Codice procedura penale.csv")

REF_MERG = DEFAULT_SAVE_DIR + ('/references_merged.csv' if isLinux else '\\references_merged.csv')
INV_LAWS_JSON = DEFAULT_SAVE_DIR + ('/invalid_laws.json' if isLinux else '\\invalid_laws.json')
DLGS_CSV = DEFAULT_SAVE_DIR + ('/dlgs.csv' if isLinux else '\\dlgs.csv')

ALL_LAWS_CSV = DEFAULT_SAVE_DIR + ("/All laws extracted.csv" if isLinux else "\\All laws extracted.csv")
ALL_LAWS_SCRAPED_JSON = DEFAULT_SAVE_DIR + ("/All laws.json" if isLinux else "\\All laws.json")

# Utility functions and constants
def write_to_file(filename, content):
    with open(filename, 'w+') as f:
        f.write(content)

def read_from_file(filename):
    with open(filename, 'r') as f:
        return f.read()
    
def read_json_file(filename):
    with open(filename, 'r') as f:
        return json.load(f)
    
def save_df_to_csv(df, filename):
    df.to_csv(filename, index=False)

# Different kind of text extraction from each type of file
def extract_text_from_pdf(pdf_path):
    text = ""
    with open(pdf_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text += page.extract_text()
    return text    

def clearCommaContent(content):
    if "<" not in content:
        return content
    
    # Check for comma class tags
    match =  re.search(r'<span class="art_text_in_comma">(.*?)</span>', comma_content, re.DOTALL)
    comma_content = match.group(1) if match else comma_content
    comma_content = re.sub(r"<div class=\"ins-akn\" eid=\"ins_\d+\">\(\(", "", comma_content, flags=re.DOTALL)
    comma_content = re.sub(r"\)\)</div>", "", comma_content, flags=re.DOTALL)
    comma_content = re.sub(r"\n", "", comma_content, flags=re.DOTALL)
    comma_content = re.sub(r"<br>", "", comma_content, flags=re.DOTALL)
    
    # Check for a tags
    aPattern = re.compile(r'<a.*?>(.*?)</a>', re.DOTALL)
    matches = aPattern.findall(content)
    if matches:
        for match in matches:
            content = re.sub(r'<a.*?>.*?</a>', match, content, count=1)

    # Check for span tags
    sPattern = re.compile(r'<span.*?>(.*?)</span>', re.DOTALL)
    matches = sPattern.findall(content)
    if matches:
        for match in matches:
            content = re.sub(r'<span.*?>.*?</span>', match, content, count=1)

    # Check for list div tags
    dlPattern = re.compile(r'<div class="pointedList-rest-akn">(.*?)</div>', re.DOTALL)
    matches = dlPattern.findall(content)
    if matches:
        for match in matches:
            content = re.sub(r'<div class="pointedList-rest-akn">.*?</div>', re.escape(match), content, count=1)
    
    # Delete remaining tags
    content = re.sub(r'<.+?>', "", content)
    
    return content

def extractCommaNumber(articleElement):
    articleElement = articleElement.strip()
    if "art." in articleElement:
        return articleElement.split(" ")[1]
    return articleElement

def extractArticleNumber(articleElement):
    match = re.search(r'n\..*?(\d+)', articleElement)
    if match:
        return int(match.group(1))
    raise Exception("Article number not found")

## Extraction of Codice Penale from website of Procura Generale Trento (**OUTDATED**)

In [ ]:
def find_articles(text):
    # Remove all sections starting with "LIBRO"
    text = re.sub(r'LIBRO.*?(?=Articolo n\.|$)', '', text, flags=re.DOTALL)

    pattern = r'(Articolo n\..*?)(\n.*?)(?=\nArticolo n\.|$)'
    matches = re.findall(pattern, text, re.DOTALL)
    return matches

text = extract_text_from_pdf(CODICE_PENALE_PDF)

matches = find_articles(text)

data = []
for article in matches:
    law_number = article[0].split("Articolo n.")[1] # Get everything after "Articolo n."
    law_text = article[1].strip()
    if '.' in law_text:
        law_title, law_text = map(str.strip, law_text.split('.', 1))
    else:
        law_title = ''
    data.append({'Source': "c.p.p.",'Law number': law_number, 'Law title': law_title, 'Law text': law_text})

df_cp = pd.DataFrame(data)
df_cp.to_csv(CODICE_PENALE_CSV, index=False)

df_cp.head()

## Extraction of Codice Penale from Normattiva

In [ ]:
class NormattivaCpScraper:
    def __init__(self, driver_path, headless=True):
        chrome_options = Options()
        if headless:
            chrome_options.add_argument("--headless")
            chrome_options.add_argument("--disable-dev-shm-usage")
            chrome_options.add_argument("--no-sandbox")
        self.service = Service(driver_path)
        self.driver = webdriver.Chrome(service=self.service, options=chrome_options)
    
    # Get the originario version of the law
    def fill_field(self, field_id, value):
        input_field = self.driver.find_element(By.ID, field_id)
        input_field.clear()
        input_field.send_keys(value)
    
    # Get the text of a specific article
    def get_cp_articles(self):
        articles_list = []
        
        # Ensure is multivigente version
        time.sleep(1)
        
        # Get articles
        albero = self.driver.find_element(By.ID, "albero")
        articles = albero.find_elements(By.CLASS_NAME, "numero_articolo")
        #print("Article len: ", len(articles))
                
        for i, article in enumerate(articles[1:]):
            if article.text.strip() != "" and article.text[0].isdigit():                
                try:
                    article.click()                    
                    time.sleep(1)
                    
                    article_title = self.driver.find_element(By.CLASS_NAME, "article-num-akn").text.strip()
                    commas = self.driver.find_elements(By.CLASS_NAME, "art-comma-div-akn")                    
                except:
                    continue
                
                #print("Comma len: ", len(commas))
                if len(commas) == 0:
                    continue
                
                page_title =  scraper.driver.title
                law_number = extractArticleNumber(page_title)
                
                firstTime = True
                finale_text = ""
                for comma in commas:
                    if firstTime:
                        time.sleep(1)
                        firstTime = False
                    
                    # Check if the content contains any multivigenza change
                    try:
                        comma_content = comma.get_attribute('outerHTML')
                    except:
                        continue
                    #print(comma_content)
                    
                    # Skip if the content contains "<span class="art_text_in_comma">((</span>" or "<span class="art_text_in_comma">))</span>"
                    if "<span class=\"art_text_in_comma\">((</span>" in comma_content or "<span class=\"art_text_in_comma\">))</span>" in comma_content:
                        continue
                    
                    try:
                        comma_number = comma.find_element(By.CLASS_NAME, "comma-num-akn").text.strip()
                    except:
                        continue
                    comma_content_element = comma.text

                    # Clear the output
                    comma_content = clearCommaContent(comma_content_element)
                    finale_text += comma_content.strip()
                articles_list.append({ "article_source": "c.p.",
                                       "article_text": finale_text,
                                       "article_number": law_number,
                                       "year": None })
                print(articles_list[-1])
            else:
                pass
#{'article_source': 'All laws', 'article_text': "...", 'year': 2021, 'article_number': 242}
        return articles_list

    # Navigate to a specific page
    def navigate_to_page(self, url):
        self.driver.get(url)
        
    # Close the driver
    def close(self):
        self.driver.quit()

scraper = NormattivaCpScraper(CHROME_DRIVERS_PATH, headless=True)
scraper.navigate_to_page("https://www.normattiva.it/uri-res/N2Ls?urn:nir:stato:regio.decreto:1930-10-19;1398")
articles = scraper.get_cp_articles()

df_cp = pd.DataFrame(articles)
df_cp.to_csv(CODICE_PENALE_CSV, index=False)

## Extraction of C.P.P. from Normattiva

In [ ]:
class NormattivaCppScraper:
    def __init__(self, driver_path, headless=True):
        chrome_options = Options()
        if headless:
            chrome_options.add_argument("--headless")
        self.service = Service(driver_path)
        self.driver = webdriver.Chrome(service=self.service, options=chrome_options)
    
    # Get the originario version of the law
    def fill_field(self, field_id, value):
        input_field = self.driver.find_element(By.ID, field_id)
        input_field.clear()
        input_field.send_keys(value)
    
    # Get the text of a specific article
    def get_cpp_articles(self):
        articles_list = []
        
        # Ensure is multivigente version
        time.sleep(1)
        
        # Get articles
        albero = self.driver.find_element(By.ID, "albero")
        articles = albero.find_elements(By.CLASS_NAME, "numero_articolo")
        #print("Article len: ", len(articles))
                
        for i, article in enumerate(articles):
            if article.text.strip() != "" and article.text[0].isdigit():                
                try:
                    article.click()                    
                    time.sleep(1)
                    
                    article_title = self.driver.find_element(By.CLASS_NAME, "article-num-akn").text.strip()
                    commas = self.driver.find_elements(By.CLASS_NAME, "art-comma-div-akn")                    
                except:
                    continue
                
                #print("Comma len: ", len(commas))
                if len(commas) == 0:
                    continue
                
                page_title =  scraper.driver.title
                law_number = extractArticleNumber(page_title)
                
                firstTime = True
                finale_text = ""
                for comma in commas:
                    if firstTime:
                        time.sleep(1)
                        firstTime = False
                    
                    # Check if the content contains any multivigenza change
                    try:
                        comma_content = comma.get_attribute('outerHTML')
                    except:
                        continue
                    #print(comma_content)
                    
                    # Skip if the content contains "<span class="art_text_in_comma">((</span>" or "<span class="art_text_in_comma">))</span>"
                    if "<span class=\"art_text_in_comma\">((</span>" in comma_content or "<span class=\"art_text_in_comma\">))</span>" in comma_content:
                        continue
                    
                    try:
                        comma_number = comma.find_element(By.CLASS_NAME, "comma-num-akn").text.strip()
                    except:
                        continue
                    comma_content_element = comma.text#find_element(By.CLASS_NAME, "art_text_in_comma")

                    # Clear the output
                    comma_content = clearCommaContent(comma_content_element)
                    finale_text += comma_content
                    
                    
                articles_list.append({ "article_source": "c.p.p.",
                                    "article_text": comma_content.strip(),
                                    "article_number": law_number,
                                    "year": None })
                print(articles_list[-1])
            else:
                pass

        return articles_list

    # Navigate to a specific page
    def navigate_to_page(self, url):
        self.driver.get(url)
        
    # Close the driver
    def close(self):
        self.driver.quit()

scraper = NormattivaCppScraper(CHROME_DRIVERS_PATH, headless=True)
scraper.navigate_to_page("https://www.normattiva.it/uri-res/N2Ls?urn:nir:stato:decreto.del.presidente.della.repubblica:1988-09-22;447")
articles = scraper.get_cpp_articles()

df_cpp = pd.DataFrame(articles)
df_cpp.to_csv(CPP_CSV, index=False)

## Extraction of Dlgs from Normattiva

In [ ]:
class NormattivaDlgsScraper:
    def __init__(self, driver_path, headless=True):
        chrome_options = Options()
        if headless:
            chrome_options.add_argument("--headless")
        self.service = Service(driver_path)
        self.driver = webdriver.Chrome(service=self.service, options=chrome_options)
    
    # Get the originario version of the law
    def fill_field(self, field_id, value):
        input_field = self.driver.find_element(By.ID, field_id)
        input_field.clear()
        input_field.send_keys(value)
    
    # Get the text of a specific article
    def get_article_text(self, numeroProvvedimento, anno, article_num=[]):
        articles_list = []
            
        self.fill_field("numeroProvvedimento", numeroProvvedimento)
        self.fill_field("annoProvvedimento", anno)

        self.driver.find_element(By.CSS_SELECTOR, "[type*='submit']").click()
        self.driver.find_elements(By.CSS_SELECTOR, "[title*='Dettaglio atto']")[0].click()
        
        time.sleep(2)
        # Ensure is multivigente version
        multivigente_button = self.driver.find_element(By.XPATH, '//a[contains(@href, "multivigenza")]')
        multivigente_button.click()
        
        # Get articles
        albero = self.driver.find_element(By.ID, "albero")
        articles = albero.find_elements(By.CLASS_NAME, "numero_articolo")
        print("Article len: ", len(articles))
                
        for article in articles:            
            if article.text.strip() != "" and article.text[0].isdigit() and "orig" not in article.text and "Allegato" not in article.text and "agg" not in article.text:
                print("In ", article.text)
                try:
                    article.click()
                except:
                    continue
                
                time.sleep(2)
                
                article_title = self.driver.find_element(By.CLASS_NAME, "article-num-akn").text.strip()
                commas = self.driver.find_elements(By.CLASS_NAME, "art-comma-div-akn")
                
                if len(commas) == 0:
                    continue
                
                page_title =  scraper.driver.title
                law_number = extractArticleNumber(page_title)
                
                firstTime = True
                finale_text = ""
                for i, comma in enumerate(commas):
                    if firstTime:
                        time.sleep(1)
                        firstTime = False
                    print(numeroProvvedimento, anno, article_title, i)
                    
                    # Check if the content contains any multivigenza change
                    comma_content = comma.get_attribute('outerHTML')
                    #print(comma_content)
                    
                    # Skip if the content contains "<span class="art_text_in_comma">((</span>" or "<span class="art_text_in_comma">))</span>"
                    if "<span class=\"art_text_in_comma\">((</span>" in comma_content or "<span class=\"art_text_in_comma\">))</span>" in comma_content:
                        continue
                    
                    try:
                        comma_number = comma.find_element(By.CLASS_NAME, "comma-num-akn").text.strip()
                    except:
                        continue
                    comma_content_element = comma.text#find_element(By.CLASS_NAME, "art_text_in_comma")

                    #print(comma_number, comma_content)
                    
                    # Clear the output
                    match =  re.search(r'<span class="art_text_in_comma">(.*?)</span>', comma_content, re.DOTALL)
                    comma_content = match.group(1) if match else comma_content
                    comma_content = re.sub(r"<div class=\"ins-akn\" eid=\"ins_\d+\">\(\(", "", comma_content, flags=re.DOTALL)
                    comma_content = re.sub(r"\)\)</div>", "", comma_content, flags=re.DOTALL)
                    comma_content = re.sub(r"\n", "", comma_content, flags=re.DOTALL)
                    comma_content = re.sub(r"<br>", "", comma_content, flags=re.DOTALL)
                    
                    comma_content = clearCommaContent(comma_content_element)
                    finale_text += comma_content.strip()
                    
                articles_list.append({ "article_source": f"{numeroProvvedimento}/{anno}".strip(),
                                       "article_text": finale_text,
                                       "article_number": law_number,
                                       "year": None })
            else:
                print("Out ", article.text)

        return articles_list

    # Navigate to a specific page
    def navigate_to_page(self, url):
        self.driver.get(url)
        
    # Close the driver
    def close(self):
        self.driver.quit()

def save_articles(articles, filename):
    df = pd.DataFrame(articles)
    df.to_csv(filename, index=False)

scraper = NormattivaDlgsScraper(CHROME_DRIVERS_PATH, headless=True)

# Check if the laws have already been scraped
scraped_laws_set = set()
if os.path.exists(ALL_LAWS_CSV):
    df_scraped = pd.read_csv(ALL_LAWS_CSV)
    first_column = df_scraped.iloc[:, 0]
    scraped_laws = df_scraped.to_dict('records')
    
    # put the already scraped laws in a set to not scrape them again
    scraped_laws_set = set(first_column)
else:
    scraped_laws = []

invalid_laws_set = set()
if os.path.exists(INV_LAWS_JSON):
    invalid_laws = json.loads(INV_LAWS_JSON)
    invalid_laws_set = set(invalid_laws)
else:
    invalid_laws = []

scraped_laws_set.update(invalid_laws_set)

df_ref = pd.read_csv(REF_MERG)
articles = []

for index, row in tqdm(df_ref.iterrows(), total=df_ref.shape[0]): # 445
    # Check if it's a D. lgs.
    if '/' not in row['Source'] or row['Source'].strip() in scraped_laws_set:
        continue
    scraped_laws_set.add(row['Source'])
    print(f"Scraping |{row['Source']}|")
    num, year = row['Source'].split("/")
            
    scraper.navigate_to_page("https://www.normattiva.it/ricerca/avanzata")
    res = scraper.get_article_text(num, year)
    if res == []:
        invalid_laws.append(f"{num}/{year}")
    else:
        articles.append(res)
    
    if articles and len(articles) % 3 == 0:
        save_articles([item for sublist in articles for item in sublist] + scraped_laws, ALL_LAWS_CSV)
        with open(INV_LAWS_JSON, 'w') as f:
            json.dumps(scraped_laws)

save_articles([item for sublist in articles for item in sublist] + scraped_laws, ALL_LAWS_CSV)
with open(INV_LAWS_JSON, 'w') as f:
    json.dumps(scraped_laws)

## Extraction of all italian's laws from Normattiva

In [ ]:
class NormattivaAllLawsScraper:
    def __init__(self, driver_path, headless=True):
        chrome_options = Options()
        if headless:
            chrome_options.add_argument("--headless")
        self.service = Service(driver_path)
        self.driver = webdriver.Chrome(service=self.service, options=chrome_options)
    
    # Get the originario version of the law
    def fill_field(self, field_id, value):
        input_field = self.driver.find_element(By.ID, field_id)
        input_field.clear()
        input_field.send_keys(value)
    
    def get_years(self):
        years = self.driver.find_elements(By.CLASS_NAME, "btn-secondary")
        time.sleep(1)
        return years
        
    # Get the text of a specific article
    def get_articles(self):
        articles_list = []
        
        # Ensure is multivigente version
        time.sleep(1)
        
        # Get articles
        albero = self.driver.find_element(By.ID, "albero")
        articles = albero.find_elements(By.CLASS_NAME, "numero_articolo")
        print("Article len: ", len(articles))
                
        for i, article in enumerate(articles[1:]):            
            print(i)
            if article.text.strip() != "" and article.text[0].isdigit():                
                try:
                    article.click()                    
                    time.sleep(1)
                    
                    article_title = self.driver.find_element(By.CLASS_NAME, "article-num-akn").text.strip()
                    commas = self.driver.find_elements(By.CLASS_NAME, "art-comma-div-akn")                    
                except:
                    continue
                
                print("Comma len: ", len(commas))
                if len(commas) == 0:
                    continue
                
                firstTime = True                
                for comma in commas:
                    if firstTime:
                        time.sleep(1)
                        firstTime = False
                    
                    # Check if the content contains any multivigenza change
                    try:
                        comma_content = comma.get_attribute('outerHTML')
                    except:
                        continue
                    #print(comma_content)
                    
                    # Skip if the content contains "<span class="art_text_in_comma">((</span>" or "<span class="art_text_in_comma">))</span>"
                    if "<span class=\"art_text_in_comma\">((</span>" in comma_content or "<span class=\"art_text_in_comma\">))</span>" in comma_content:
                        continue
                    
                    try:
                        comma_number = comma.find_element(By.CLASS_NAME, "comma-num-akn").text.strip()
                    except:
                        continue
                    comma_content_element = comma.text#find_element(By.CLASS_NAME, "art_text_in_comma")

                    # Clear the output
                    comma_content = clearCommaContent(comma_content_element)
                    
                    print(article_title, comma_number, comma_content)
                    
                    articles_list.append({ "Source": "All laws",
                                            "Article": article_title,
                                            "Comma number": comma_number,
                                            "Comma content": comma_content.strip()}) # Numeration not working in case of -bis... extract
            else:
                print("Out ", article.text)

        return articles_list

    # Navigate to a specific page
    def navigate_to_page(self, url):
        self.driver.get(url)
        
    # Close the driver
    def close(self):
        self.driver.quit()

scraper = NormattivaAllLawsScraper(CHROME_DRIVERS_PATH, headless=False)
scraper.navigate_to_page("https://www.normattiva.it/ricerca/elencoPerData")
years = scraper.get_years()
data = []

for year in range(1861, 2025):
    scraper.driver.get("https://www.normattiva.it/ricerca/avanzata")
    scraper.fill_field("annoProvvedimento", year)
    scraper.driver.find_element(By.CSS_SELECTOR, "[type*='submit']").click()
    time.sleep(0.5)
    
    validPage = True
    curr_page = 1
    law_urls = []
    
    while validPage:
        try:
            page = year.find_element_by_xpath("//a[contains(text(), '{curr_page}')]")
            curr_page += 1
        except:
            validPage = False
            continue
        
        laws = year.find_element_by_xpath('//a[@title="Dettaglio atto"]')
        
        for law in laws:
            law.click()
            articles = scraper.get_articles()
            
            data.append(articles)    

df_all_laws = pd.DataFrame(data)
df_all_laws.head()

## Post all Dlgs extraction -> Merge previous df with the laws.csv file

In [ ]:
#df_laws = pd.read_csv(LAWS_CSV)
df_cp = pd.read_csv(CODICE_PENALE_CSV)
df_cpp = pd.read_csv(CPP_CSV)
#df_dlgs = pd.read_csv(LAWS_CSV)
final_df = pd.concat([df_cp, df_cpp], ignore_index=True)

In [ ]:
import json
with open('/home/antonelli2/Thesis/Documents/Generated/invalid_laws.json', 'r') as file:
    data = json.load(file)